In [1]:
import pandas as pd
from simpledbf import Dbf5
import numpy as np
from os import listdir
from os.path import isfile, join
from functions.cargar_data import *

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [25]:
# carga personas
personas96 = pd.read_csv('marcos_censales/peronas_zona_96.csv')
personas96.head(2)

,codcomp,personas
0,1001003001,10
1,1001003002,19


In [2]:
path = 'datos_ine_96/dbfs/'

In [3]:
# lee datos de Mvdeo.
dbf = Dbf5(path + '01 idgeo 96-04 mvd.dbf')
mvo = dbf.to_dataframe()
print(mvo.shape)
mvo.head(2)

(12165, 4)


,CCOMP96,CCOMP04,REG96A04,MODIF96A04
0,101001001,101001001,0,SIN MODIFICACION
1,101001003,101001003,0,SIN MODIFICACION


In [4]:
mvo.groupby('MODIF96A04').size()

MODIF96A04
FUSION               240
PARTICION           3153
SIN MODIFICACION    8772
dtype: int64

In [5]:
# arregla typo en data de Flores
dbf = Dbf5(path + '07 idgeo 96-04 flores.dbf')
flores = dbf.to_dataframe().rename({'CCOPM04': 'CCOMP04'}, axis=1)
print(flores.shape)
flores.head(2)

(724, 6)


,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04
0,701001001,701001001,0,SIN MODIFICACION,7320,7320
1,701001002,701001002,0,SIN MODIFICACION,7320,7320


In [8]:
# lee, enlista y convierte a DF
files = [f for f in listdir(path) if isfile(join(path, f))]
files.remove('01 idgeo 96-04 mvd.dbf')
files.remove('07 idgeo 96-04 flores.dbf')
df_list = [(Dbf5(path + i).to_dataframe()) for i in files]
df_list.append(flores)
dfs = pd.concat(df_list)
print('Con Flores: ', dfs.shape[0])
dfs.head(5)

Con Flores:  48571


,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04
0,201001001,201001001,0,SIN MODIFICACION,2220,2220.0
1,201001002,201001002,0,SIN MODIFICACION,2220,2220.0
2,201001003,201001003,0,SIN MODIFICACION,2220,2220.0
3,201001004,201001004,0,SIN MODIFICACION,2220,2220.0
4,201001005,201001005,0,SIN MODIFICACION,2220,2220.0


In [34]:
# exporta
dfs.to_csv('datos_ine_96/correspondencia_interior.csv', index=False, float_format='%.0f')
mvo.to_csv('datos_ine_96/correspondencia_mvdeo.csv', index=False)

In [9]:
dfs.groupby('MODIF96A04').size()

MODIF96A04
FUSION                959
Inf 2 de CPV96          5
PARTICION            6613
SIN MODIFICACION    40994
dtype: int64

In [21]:
dfs.head()

,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04
0,201001001,201001001,0,SIN MODIFICACION,2220,2220.0
1,201001002,201001002,0,SIN MODIFICACION,2220,2220.0
2,201001003,201001003,0,SIN MODIFICACION,2220,2220.0
3,201001004,201001004,0,SIN MODIFICACION,2220,2220.0
4,201001005,201001005,0,SIN MODIFICACION,2220,2220.0


In [28]:
dfs_personas = dfs.merge(personas96, left_on='CCOMP96', right_on='codcomp')
dfs_personas.head(2)

,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04,codcomp,personas
0,201001001,201001001,0,SIN MODIFICACION,2220,2220.0,201001001,144
1,201001002,201001002,0,SIN MODIFICACION,2220,2220.0,201001002,133


In [31]:
dfs_personas_sd = dfs_personas.drop_duplicates('CCOMP96')
dfs_personas_sd.head()

,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04,codcomp,personas
0,201001001,201001001,0,SIN MODIFICACION,2220,2220.0,201001001,144
1,201001002,201001002,0,SIN MODIFICACION,2220,2220.0,201001002,133
2,201001003,201001003,0,SIN MODIFICACION,2220,2220.0,201001003,147
3,201001004,201001004,0,SIN MODIFICACION,2220,2220.0,201001004,123
4,201001005,201001005,0,SIN MODIFICACION,2220,2220.0,201001005,87


In [30]:
parti = dfs.loc[dfs.MODIF96A04=='PARTICION']
parti.head(6)

,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04
132,201009021,201009801,1,PARTICION,2220,2220.0
133,201009021,201009802,1,PARTICION,2220,2220.0
134,201009021,201009803,1,PARTICION,2220,2220.0
135,201009021,201009804,1,PARTICION,2220,2220.0
136,201009021,201009805,1,PARTICION,2220,2220.0
137,201009020,201009806,1,PARTICION,2220,2220.0


In [46]:
parti_group = parti.groupby('CCOMP96').size().to_frame(name='division')
parti_group.head()

,division
CCOMP96,
201009020,7
201009021,5
201009023,2
201016010,2
201017002,3


In [47]:
parti_group.merge(dfs_personas_sd, left_on='CCOMP96', right_on='codcomp')

,division,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04,codcomp,personas
0,7,201009020,201009806,1,PARTICION,2220,2220.0,201009020,124
1,5,201009021,201009801,1,PARTICION,2220,2220.0,201009021,247
2,2,201009023,201009814,1,PARTICION,2220,2220.0,201009023,86
3,2,201016010,201016010,1,PARTICION,2220,2220.0,201016010,31
4,3,201017002,201017800,1,PARTICION,2220,2220.0,201017002,24
...,...,...,...,...,...,...,...,...,...
1417,5,1907202002,1907202800,1,PARTICION,19723,19723.0,1907202002,0
1418,7,1907202035,1907202805,1,PARTICION,19723,19723.0,1907202035,57
1419,2,1909103001,1909103500,1,PARTICION,19522,19522.0,1909103001,4
1420,2,1909105001,1909205800,1,PARTICION,19900,19522.0,1909105001,58
